In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import torch
import transformers
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

import accelerate
from accelerate import Accelerator

import langchain

In [2]:
from transformers import T5Tokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler

In [3]:
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader

import peft
from peft import PeftModel

c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\exllama.py:12: UserWarning: AutoAWQ could not load ExLlama kernels extension. Details: DLL load failed while importing exl_ext: 지정된 프로시저를 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load ExLlama kernels extension. Details: {ex}")
c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\exllamav2.py:13: UserWarning: AutoAWQ could not load ExLlamaV2 kernels extension. Details: DLL load failed while importing exlv2_ext: 지정된 프로시저를 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load ExLlamaV2 kernels extension. Details: {ex}")
c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\gemm.py:14: UserWarning: AutoAWQ could not load GEMM kernels extension. Details: DLL load failed while importing awq_ext: 지정된 프로시저를 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load GEMM kernels extension. Details: {ex}")
c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\gemv.py:11: UserWarning: AutoAWQ could not

In [4]:
import datasets
from datasets import Dataset
from transformers import Trainer, TrainingArguments

In [5]:
from transformers import T5ForTokenClassification

In [6]:
from torch import nn

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
model = T5ForTokenClassification.from_pretrained(
    pretrained_model_name_or_path = 't5-small'
).to(device)

Some weights of T5ForTokenClassification were not initialized from the model checkpoint at t5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
epochs = 5
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5ForTokenClassification.from_pretrained(
    pretrained_model_name_or_path='t5-small',
)

Some weights of T5ForTokenClassification were not initialized from the model checkpoint at t5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
data = pd.read_csv("train.csv")

In [11]:
df = data['first_party'] + data['second_party'] + data['facts'] 

In [12]:
df = pd.DataFrame(df)
df = pd.concat([df, data['first_party_winner']], axis=1)

In [13]:
df.columns = ['informations', 'label']

In [14]:
def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds.cpu().numpy(), axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [15]:
def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text = data.informations.tolist(),
        padding= 'longest',
        truncation = True,
        return_tensors = 'pt'
    )
    input_ids = tokenized['input_ids'].to(device)
    attention_mask = tokenized['attention_mask'].to(device)
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    return TensorDataset(input_ids, attention_mask, labels)

In [16]:
def get_dataloader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler = data_sampler, batch_size = batch_size)
    return dataloader

In [17]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch import optim
from transformers import BertForSequenceClassification
from torch import nn
import math
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [18]:
epochs = 5
batch_size = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-multilingual-cased',
    do_lower_case = False
)

c:\users\dohyeong\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6*len(df)), int(0.8*len(df))]
)

c:\users\dohyeong\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [20]:
train

,informations,label
1753,Gerald E. MansellGaye M. Mansell Major Gerald ...,1
259,FreedmanMarylandMaryland required that all fil...,1
2072,"BMW of North America, Inc.GoreAfter purchasing...",1
1000,MarylandWirtzThe Fair Labor Standards Act of 1...,0
56,Randall D. WhiteState of IllinoisDuring Randal...,0
...,...,...
1402,"Zenith Radio CorporationHazeltine Research, In...",1
2018,BlessingFreestoneCathy Freestone and four othe...,1
2240,United States Trust Company of New YorkNew Jer...,1
1964,"Commissioner of Internal Revenue, Philip D. Fa...",0


In [21]:
valid

,informations,label
1314,"Tim Shoop, WardenDanny HillIn 1986, Danny Hill...",1
1736,"Gloria Gail Kurns, Executrix of the Estate of ...",0
269,United StatesAnthony James KebodeauxAnthony Ke...,1
1749,Seila Law LLCConsumer Financial Protection Bur...,1
2044,"United States of AmericaRene Sanchez-Gomez, et...",1
...,...,...
1253,Ivan EberhartUnited StatesIvan Eberhart was co...,1
1276,Pearly L. WilsonRichard Seiter et al.While det...,0
912,"Steven A. LevinUnited States, et al.On March 1...",1
491,"American Tradition Partnership, Inc.Steve Bull...",1


In [22]:
train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_dataloader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_dataloader(valid_dataset, RandomSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_dataloader(test_dataset, RandomSampler, batch_size)

In [23]:
model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path='bert-base-multilingual-cased',
    num_labels = 2
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0
    
    for input_ids, attention_mask, labels in dataloader:
        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )
        
        loss = outputs.loss
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss = train_loss / len(dataloader)
    return train_loss

In [26]:
def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        criterion = nn.CrossEntropyLoss()
        val_loss, val_accuracy = 0.0, 0.0
        
        for input_ids, attention_mask, labels in dataloader:
            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                labels = labels
            )
            logits = outputs.logits
            
            loss = criterion(logits, labels)
            logtis = logits.detach().cpu().numpy()
            labels_ids = labels.to('cpu').numpy()
            accuracy = calc_accuracy(logits, labels_ids)
            
            val_loss += loss
            val_accuracy += accuracy
            
        val_loss = val_loss/len(dataloader)
        val_accuracy = val_accuracy / len(dataloader)
        return val_loss, val_accuracy

In [27]:
best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f'Epoch: {epoch+1} train loss: {train_loss:.4f} val loss: {val_loss:.4f} val_acc: {val_accuracy:.4f}')
    
    if val_loss < best_loss:
        best_loss = val_loss

c:\users\dohyeong\anaconda3\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch: 1 train loss: 0.6460 val loss: 0.6463 val_acc: 0.6512
Epoch: 2 train loss: 0.6371 val loss: 0.6466 val_acc: 0.6512
Epoch: 3 train loss: 0.6322 val loss: 0.6500 val_acc: 0.6512
Epoch: 4 train loss: 0.6122 val loss: 0.6508 val_acc: 0.6472
Epoch: 5 train loss: 0.5832 val loss: 0.7463 val_acc: 0.6089


In [28]:
test_loss, test_score = evaluation(model, test_dataloader)

In [29]:
test_loss

tensor(0.7079, device='cuda:0')

In [30]:
test_score

0.6491935483870968